# Finding the ground state via imaginary time evolution

## Physics

### Quantum harmonic oscillator

Consider the quantum harmonic oscillator (QHO) - the quantum-mechanical analog
of the classical harmonic oscillator. The Hamiltonian is defined as 

$$
H = \hat T + \hat V = \frac{\hat p^2}{2m} + \frac{1}{2}m w^2\hat x^2
$$

where $\hat p$ is the momentum operator, $m$ is the mass, $\omega$ is the
angular frequency, and $\hat x$ is the position operator.

Our goal is to find the ground state of a one-dimensional QHO for a given
angular frequency $\omega_x$. For a given initial state the imaginary time
evolution can be iteratively applied to evolve the initial state to the ground
state of our system. 

### The imaginary time evolution
The imaginary time evolution is equivalent to applying the split-step operator

$$
e^{-\tfrac{i}{\hbar}H \mathrm{dt}} \approx e^{-\tfrac{i}{\hbar}\frac{\hat T}{2} \mathrm{dt}} e^{-\tfrac{i}{\hbar}\hat V \mathrm{dt}} e^{-\tfrac{i}{\hbar}\frac{\hat T}{2} \mathrm{dt}}
$$

with imaginary time step $\mathrm{dt}=-i\mathrm{dt}$. 
By exchanging the time step, the time evolution operator turns into a
real-valued coefficient. Expanding the initial state in terms of eigenstates of
the system $\Psi = \sum_n a_n\Psi_n$ reveals that each eigenstate $\Psi_n$ is
scaled by $e^{-\frac{1}{\hbar} E_n \mathrm{dt}}a_n$ where $H \Psi_n = E_n
\Psi_n$. Thus, by iteratively applying the split-step operator, an approximation
for the ground state will remain. States with higher energy will be suppressed
by their small coefficient. Note that the wavefunction has to be normalized
after each application as the split-step operator became non-unitary. 
Spoiler alert: this is covered inside the `split_step` function if the argument
`is_complex` is set to `True`.

## Code

In [9]:
# math lib for computation on CPU/GPU/TPU
import jax.numpy as jnp
from jax.lax import scan
# fftarray
from fftarray.backends.jax_backend import JaxTensorLib
from fftarray.backends.np_backend import NumpyTensorLib
from fftarray.fft_constraint_solver import fft_dim_from_constraints
# matterwave
from matterwave import split_step, get_ground_state_ho, get_e_kin

# constants
from scipy.constants import pi, hbar, Boltzmann
from matterwave.rb87 import m as m_rb87

### Initialize global variables

In [4]:
# here: mass of rb87
mass = m_rb87 # kg
# the angular frequency of the QHO whose ground state and energy is to find:
omega_x = 2.*pi # Hz
# omega_x_init is used to generate the initial state that will be evolved to the
# desired ground state (the generated state is the ground state of the QHO with
# angular frequency omega_x_init):
omega_x_init = 2.*pi*0.1 # Hz
# time step for split-step:
dt = 1e-4 # s
# choose dimension x [m]

### Initialize the wavefunction

In [13]:
# insert the constraints for the x dimension, all other free variables will be
# set accordingly
x_dim = fft_dim_from_constraints(
    "x",
    n = 2048,
    pos_min = -200e-6,
    pos_max = 200e-6,
    freq_middle = 0.
)
# initialize the wavefunction as the ground state of the QHO with omega_x_init
wf_init = get_ground_state_ho(
    dim = x_dim,
    tlib = JaxTensorLib(),
    omega = omega_x_init,
    mass = mass
)

AssertionError: 